# Requirements

In [1]:
import functools

# Problem

Given a set of coin values $C$, e.g., $\{1, 2, 5\}$ and an amount of money $m$, find weights such that
$$
m = \sum_{i} w_i c_i
$$
and the sum of weights $\sum_i w_i$ is minimal.  Weights are positive integers.

# Implementations

Various implementations are possible.

## Greedy algorithms

In [2]:
def change_greedy(amount, input_coins):
    coins = sorted(input_coins, reverse=True)
    change = []
    for coin in coins:
        change.append(amount//coin)
        amount %= coin
    return dict(zip(coins, change))

In [3]:
change_greedy(23, [1, 2, 5])

{5: 4, 2: 1, 1: 1}

This works, however, not for all sets of coins.  Consider, e.g., $\{1, 3, 4\}$ to give change for 6.

In [4]:
change_greedy(6, [1, 3, 4])

{4: 1, 3: 0, 1: 2}

Although the change is correct, it is not minimal since you could use two coins of 3.

## Recursive algorithm

In [5]:
def change_recursive(amount, input_coins):
    coins = sorted(input_coins, reverse=True)
    def compute(amount, coins):
        if len(coins) == 1:
            return [amount//coins[0]]
        best_change = [amount + 1]
        for nr_coins in range(1 + amount//coins[0]):
            change = [nr_coins]
            remainder = amount - nr_coins*coins[0]
            change.extend(compute(remainder, coins[1:]))
            if sum(change) < sum(best_change):
                best_change = change
        return best_change
    return dict(zip(coins, compute(amount, coins)))

In [6]:
change_recursive(23, [1, 2, 5])

{5: 4, 2: 1, 1: 1}

In [7]:
change_recursive(6, [1, 3, 4])

{4: 0, 3: 2, 1: 0}

Although it is correct, it is extremely slow.  In fact, it is $O(2^m)$.

In [8]:
%timeit change_recursive(123, [1, 2, 5])

388 µs ± 4.22 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
%timeit change_recursive(1_234, [1, 2, 5])

39.3 ms ± 623 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%timeit change_greedy(1_234, [1, 2, 5])

765 ns ± 6.53 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Memoization

Typically, memoization can help solve this kind of performance problem.  We can use the `lrucache` from `functools`.

In [11]:
def change_memoized(amount, coins):
    coins = tuple(sorted(coins, reverse=True))
    @functools.lru_cache
    def compute(amount, coins):
        if len(coins) == 1:
            return [amount//coins[0]]
        best_change = [amount + 1]
        for nr_coins in range(1 + amount//coins[0]):
            change = [nr_coins]
            remainder = amount - nr_coins*coins[0]
            change.extend(compute(remainder, coins[1:]))
            if sum(change) < sum(best_change):
                best_change = change
        return best_change
    return dict(zip(coins, compute(amount, coins)))

In [12]:
change_memoized(23, [1, 2, 5])

{5: 4, 2: 1, 1: 1}

However, it doesn't in this case, so a hand-tuned implementation would be required.

In [13]:
%timeit change_recursive(1_234, [1, 2, 5])

38.8 ms ± 501 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%timeit change_memoized(1_234, [1, 2, 5])

47.5 ms ± 1.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Dynamic progamming

For this type of problem, dynamic programming is a nice solution.  The complexity of this algorithm is $O(m|C|)$, so linear in $m$ rather than exponential.

In [15]:
def change_dynamic_programming(amount, input_coins):
    coins = sorted(input_coins)
    table = [list(range(amount + 1))]
    for coin in coins[1:]:
        if coin > amount:
            break
        row = table[-1][:coin]
        row.append(1)
        for value in range(coin + 1, amount + 1):
            row.append(min(table[-1][value], 1 + row[value - coin]))
        table.append(row)
    change = {coin: 0 for coin in coins}
    i, j = len(table) - 1, len(table[-1]) - 1
    while table[i][j] > 0:
        if i > 0 and table[i][j] == table[i - 1][j]:
            i -= 1
        elif j >= coins[i]:
            change[coins[i]] += 1
            j -= coins[i]
    return change

In [16]:
change_dynamic_programming(23, [1, 2, 5])

{1: 1, 2: 1, 5: 4}

In [17]:
change_dynamic_programming(6, [1, 3, 4])

{1: 0, 3: 2, 4: 0}

In [18]:
%timeit change_recursive(1_234, [1, 2, 5])

38.3 ms ± 412 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
%timeit change_dynamic_programming(1_234, [1, 2, 5])

563 µs ± 4.12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [20]:
%timeit change_greedy(1_234, [1, 2, 5])

761 ns ± 9.45 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Dynamic programming is of course slower than the greedy algorithm, but it produces the correct result for weird coinage.

# Verification

For the coinage $\{1, 2, 5\}$ all three algoritmns should produce the same result.

In [21]:
coins = [1, 2, 5]
failures = 0
for amount in range(1, 25):
    target = change_greedy(amount, coins)
    for func in (change_recursive, change_memoized, change_dynamic_programming):
        if func(amount, coins) != target:
            print(f'failure for {amount} using {func.name}')
            failures += 1
print(f'{failures} failures')

0 failures


Of course, dynamic programming and the recursive algorithm should yield the same result.

In [22]:
coins = [1, 3, 4]
failures = 0
for amount in range(1, 101):
    if change_recursive(amount, coins) != change_dynamic_programming(amount, coins):
        print(amount)
        failures += 1
print(f'{failures} failures')

0 failures


Out of curiosity, lets see how often dynamic programming deviates from the greedy algorithm.

In [23]:
coins = [1, 3, 4]
failures = 0
for amount in range(1, 101):
    if change_greedy(amount, coins) != change_dynamic_programming(amount, coins):
        print(amount)
        failures += 1
print(f'{failures} failures')

6
9
10
13
14
17
18
21
22
25
26
29
30
33
34
37
38
41
42
45
46
49
50
53
54
57
58
61
62
65
66
69
70
73
74
77
78
81
82
85
86
89
90
93
94
97
98
47 failures
